In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pprint

import numpy as np


pp = pprint.PrettyPrinter()

def get_meshgrid(X, Y, R, n_x=1000, n_y=1000):
    Rx = R
    Ry = (n_y / n_x) * Rx
    x_ubd, x_lbd = X + Rx, X - Rx
    y_ubd, y_lbd = Y + Ry, Y - Ry
    
    x_rng = np.linspace(x_lbd, x_ubd, n_x, dtype=np.float64)
    y_rng = np.linspace(y_lbd, y_ubd, n_y, dtype=np.float64)
    
    X, Y = np.meshgrid(x_rng, y_rng)
    Z = X + 1j*Y
    return Z

def fractal_to_img(a):
    # cyclically color the points by number of iterations to diverge
    cycle_steps = 20 # number of steps before colors cycle
    a_cyclic = 2 * np.pi * a / cycle_steps
    rgb = [127 + 80 * np.cos(a_cyclic),
           127 + 80 * np.sin(a_cyclic),
           127 + 80 * np.cos(a_cyclic)]
    img = np.stack(rgb, 2)

    # color points which never diverged black
    img[a == a.max()] = 0
    img = np.uint8(img)
    return img

In [ ]:
import os

from scipy.misc import imsave
import tensorflow as tf

from utils import pp, get_meshgrid, fractal_to_img

X = 0.0
Y = 0.0
R = 2.0
output_width = 1000
output_height = 1000

def main(_):
    pp.pprint(FLAGS.__flags)
    
    if not os.path.exists(FLAGS.img_dir):
        os.mkdir(FLAGS.img_dir)
    
    # get meshgrid over which to iterate to produce fractal
    Z = get_meshgrid(FLAGS.X, FLAGS.Y, FLAGS.R,
                     n_x=FLAGS.output_width, n_y=FLAGS.output_height)
    
    # build computational graph
    xs = tf.constant(Z, name='xs')
    zs = tf.Variable(xs, name='zs')
    ns = tf.Variable(tf.zeros_like(xs, tf.float32), name='ns')
    
    zs_ = zs * zs + xs
    not_diverged = tf.abs(zs_) < 4
    update_zs = zs.assign(zs_)
    update_ns = ns.assign_add(tf.cast(not_diverged, tf.float32)) # add one for each iteration that the point did not diverge
    update_step = tf.group(update_zs, update_ns)
    
    init_op = tf.global_variables_initializer()
    
    # run num_iters iterations to generate non-divergence counts for each pixel location
    with tf.Session() as sess:
        init_op.run()
        for i in range(FLAGS.num_iters):
            update_step.run()
        ns_np = ns.eval()
    
    img = fractal_to_img(ns_np)
    img_name = 'mandelbrot_%f-%f-%f' % (FLAGS.X, FLAGS.Y, FLAGS.R)
    imsave('%s/%s.png' % (FLAGS.img_dir, img_name), img)

if __name__ == '__main__':
    tf.app.run()

Do the following:

`git config --global user.name`